# 北京女子图鉴

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from pyecharts import Bar
from pyecharts import online

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# 让 pyecharts 在 jupyter 下载时有图片
online()

In [ ]:
# 获取优酷评论方法
def get_youku_comments():
    pass

# 获取优酷弹幕方法
def get_youku_dm():
    pass

In [ ]:
import base64
 
char = "\xc0\xc4\xc8\xcc\xd0\xd4\xd8\xdc\xe0\xe4"
char_map = {char[i]: str(i) for i in range(len(char))}
 
# suffix = "wy024"
suffix = ""
suffix_map = {suffix[i]: str(i*2) for i in range(len(suffix))}
 
def uid2eid(uid):
    """ 用户数字ID 加密为 字符串ID"""
    return encrypt(uid, prefix="P")
 
def uid2vid(uid):
    """ 视频数字ID 加密为 字符串ID """
    return encrypt(uid, prefix="\\")
 
def eid2uid(eid):
    """ 字符串ID 解密为 数字ID """
    # 判断最后一位是否需要进行fix
    fix = suffix_map[eid[-1]] if eid[-1] in suffix else ""
    # 去掉最后一位
    eid = eid[:-1]
    ss = base64.b64decode(eid)[1:]
    num = []
    for s in ss:
        num.append(char_map[s])
 
    num.append(fix)
    uid = int("".join(num)) / 4
    return uid
 
def encrypt(id, prefix):
    num = str(int(id)*4)
    fix = "="
    if len(num) % 3 == 0:
        fix = suffix[int(num[-1])/2]
        num = num[:-1]
 
    ss = prefix
    for s in str(num):
        ss += char[int(s)]
 
    eid = base64.b64encode(ss) + fix
    return eid

## 优酷相关

### 评论
评论 URL：
http://p.comments.youku.com/ycp/comment/pc/commentList?app=100-DDwODVkv&objectId=884389334&objectType=1&listType=0&currentPage=6&pageSize=30&sign=2056692c183efe4254a20c2e431946a3&time=1525611949

- objectId：视频 ID
- currentPage：评论当前页面
- pageSize：页面评论数量

**返回的数据**
```json
{
    "code": 0, 
    "data": {
        "sourceCommentSize": 629, 
        "totalSize": 837, 
        "totalPage": 21, 
        "pageSize": 30, 
        "comment": [
            {
                "atUsers": { }, 
                "flags": { }, 
                "upCount": 8, 
                "parentCommentId": 0, 
                "userId": 1155192313, 
                "content": "感觉最后会跟卢家凯在一起", 
                "downCount": 0, 
                "replyCount": 0, 
                "createTime": 1524649229889, 
                "userIsLiked": 0, 
                "parentComment": { }, 
                "id": 3409500200, 
                "user": {
                    "vipInfo": {}, 
                    "userLevel": 17, 
                    "avatarMiddle": "https://wwc.alicdn.com/avatar/getAvatar.do?userId=859825765&width=160&height=160&type=sns", 
                    "avatarLarge": "https://wwc.alicdn.com/avatar/getAvatar.do?userId=859825765&width=160&height=160&type=sns", 
                    "userName": "n小允", 
                    "userId": 1155192313, 
                    "userCode": "UNDYyMDc2OTI1Mg==", 
                    "avatarSmall": "https://wwc.alicdn.com/avatar/getAvatar.do?userId=859825765&width=160&height=160&type=sns"
                }
            }
        ], 
        "currentPage": 6, 
        "hot": [ ]
    }
}
```

### 弹幕
弹幕查询地址：http://service.danmu.youku.com/list?mat=30&mcount=1&ct=1001&iid=884389334&aid=329435&cid=97&lid=0&ouid=0

参数：
- mat：第几次请求弹幕（mat 需要根据实际的弹幕量进行调整）
- iid：视频 ID

返回数据格式：
```json
{
    "count": 16, 
    "filtered": 1, 
    "result": [
        {
            "aid": 329435, 
            "content": "单身的话还是会去北京", 
            "createtime": 1524581256000, 
            "ct": 3002, 
            "id": 707049746, 
            "iid": 884389334, 
            "ipaddr": 1740701100, 
            "level": 1, 
            "lid": 0, 
            "mat": 30, 
            "ouid": 685162531, 
            "playat": 1800214, 
            "propertis": "{\"pos\":3,\"size\":1,\"effect\":0,\"color\":15060379,\"color2\":12098412}", 
            "status": 99, 
            "type": 1, 
            "uid": 685162531, 
            "ver": 1
        }
    ]
}
```

## 解析代码

### 弹幕

In [ ]:
"""
请求每一集的弹幕
danmu_csv: 保存到 csv 文件的文件流
vid: 视频 id
episode_cn: 当前集的中文名
"""
def get_youku_danmu(danmu_csv,vid,episode_cn):
    MAT = 1
    FLAG = True
    while FLAG:
        print(episode_cn,"----", MAT)
        url = "http://service.danmu.youku.com/list?mat="+str(MAT)+"&mcount=1&ct=1001&iid="+vid+"&aid=329435&cid=97&lid=0&ouid=0"
        data = requests.get(url).json()
        if len(data["result"]) > 0:
            # 解析数据
            parse_youku_danmu(danmu_csv,data["result"],episode_cn)
            MAT += 1
            time.sleep(1)
        else:
            FLAG = False
            
"""
弹幕数据解析
"""
def parse_youku_danmu(danmu_csv,danmus,episode_cn):
    for danmu in danmus:
        uid = danmu["uid"]
        ipaddr = danmu["ipaddr"]
        episode_id = danmu["iid"]
        createtime = danmu["createtime"]
        content = danmu["content"]
        danmu_csv.writerow((uid,ipaddr,episode_id,episode_cn,createtime,content))

def youku_danmu(filepath,episodes):
    danmu = open(filepath,"a")
    danmu_csv = csv.writer(danmu)
    danmu_csv.writerow(["uid","ipaddr","episode_id","episode_cn","createtime","content"])
    
    for i,episode in (enumerate(episodes)):
        get_youku_danmu(danmu_csv,episode[0],episode[1])
    
    danmu.close()

In [ ]:
bjnztj =[("880078209","第 01 集"),("880078104","第 02 集"),("880078056","第 03 集"),("880109272","第 04 集"),("880112230","第 05 集"),\
         ("880112798","第 06 集"),("880143160","第 07 集"),("880144723","第 08 集"),("880145085","第 09 集"),("880179270","第 10 集"),\
         ("880179884","第 11 集"),("884389334","第 12 集"),("884389588","第 13 集"),("888106992","第 14 集"),("888106801","第 15 集"),\
         ("888107530","第 16 集"),("888151519","第 17 集"),("888152909","第 18 集"),("888153384","第 19 集"),("888176615","第 20 集")]


In [ ]:
bjnztj_file = "/Users/weduoo/Desktop/youku/bjnztj_danmu.csv"
youku_danmu(bjnztj_file,bjnztj)

### 评论

In [ ]:
"""
请求每一集的评论
danmu_csv: 保存到 csv 文件的文件流
vid: 视频 id
episode_cn: 当前集的中文名
"""
def get_youku_comments(comment_csv,vid,episode_cn):
    PAGE = 1
    FLAG = True
    while FLAG:
        print(episode_cn,"----", PAGE)
        url = "http://p.comments.youku.com/ycp/comment/pc/commentList?app=100-DDwODVkv&objectId="+vid+"&objectType=1&listType=0&currentPage="+str(PAGE)+"&pageSize=30&sign=c8c7a7ab7056176b3b28ebdd27769a21&time=1525962252"
        data = requests.get(url).json()
        if len(data["data"]["comment"]) > 0:
            # 解析数据
            parse_youku_comments(comment_csv,data["data"]["comment"],vid,episode_cn)
            PAGE += 1
            time.sleep(1)
        else:
            FLAG = False
            
"""
评论数据解析
"""
def parse_youku_comments(comment_csv,comments,vid,episode_cn):
    for comment in comments:
        userId = comment["user"]["userId"]
        userName = comment["user"]["userName"]
        createTime = comment["createTime"]
        upCount = comment["upCount"]
        downCount = comment["downCount"]
        content = comment["content"].replace("\n","")
        comment_csv.writerow((userId,userName,vid,episode_cn,createTime,upCount,downCount,content))

def youku_comments(filepath,episodes):
    comment = open(filepath,"a")
    comment_csv = csv.writer(comment)
    comment_csv.writerow(["userId","userName","episode_id","episode_cn","createTime","upCount","downCount","content"])
    
    for i,episode in (enumerate(episodes)):
        get_youku_comments(comment_csv,episode[0],episode[1])
    
    comment.close()

In [ ]:
bjnztj_cmments_file = "/Users/weduoo/Desktop/youku/bjnztj_comments.csv"
youku_comments(bjnztj_cmments_file,bjnztj)

## 弹幕数据分析

In [ ]:
bjnvtj_danmu = pd.read_csv("/Users/weduoo/Desktop/youku/bjnztj_danmu.csv")
bjnvtj_danmu.head()

In [ ]:
ips = pd.read_csv("/Users/weduoo/Desktop/ip_cn.csv")
ips.head()

In [ ]:
def convert_ip(ip_list):
    ip_city = []
    for ip in tqdm(ip_list):
        city = ips[(ip >= ips["int_start"] ) & (ip <=ips["int_end"])]
        if len(city) > 0:
            ip_city.append(city["sheng"].values[0])
        else:
            ip_city.append("局域")
    return ip_city

In [ ]:
bjnvtj_danmu["ip_city"] = convert_ip(list(bjnvtj_danmu["ipaddr"]))
bjnvtj_danmu.head()

In [ ]:
from pyecharts import Pie
bjnvtj_city = bjnvtj_danmu.groupby("ip_city").size()
bjnvtj_city_x = list(bjnvtj_city.index)
bjnvtj_city_y = list(bjnvtj_city.values)
pie = Pie("甄嬛传评论数量地区占比",title_pos='center',height=550)
pie.add("", bjnvtj_city_x, bjnvtj_city_y, is_label_show=True,legend_pos='left',is_legend_show=False,radius=[45, 70])
pie

In [ ]:
bar = Bar("北京女子图鉴地区占比",title_pos='center',height=550,width=900)
bar.add("", bjnvtj_city_x, bjnvtj_city_y, is_label_show=True,legend_pos='left',is_legend_show=False)
bar

In [ ]:
from pyecharts import Map
map = Map("地区分布",title_pos='center', width=800, height=600)
map.add("", bjnvtj_city_x, np.array(bjnvtj_city_y)/100, maptype='china', is_visualmap=True,is_label_show=True,
        visual_text_color='#000')
map

In [ ]:
def save_wordcloud(text,path,name):
    # 用于支持中文的设置
    font = '/Library/Fonts/Songti.ttc'
    wordcloud = WordCloud(background_color="white",font_path=font,\
                          width=1000, height=1000, margin=2).generate(text)
    # 生成图片
    plt.figure(figsize=(14,14))
    plt.imshow(wordcloud)
    plt.axis("off")
#     plt.show()
    wordcloud.to_file(path + name + ".png")

In [ ]:
def remove_words(txt):
    words = ["哈","呵","啦","啊","陈可","杨大赫","张超","卢家凯","朱朝阳","吴昊","向荣","于扬","许斯明",
             "王涛","何志","黄越彬","无语","愤怒"]
    for word in words:
        txt = txt.replace(word,"")
    return txt

In [ ]:
def city_wordcloud(path):
    citys = bjnvtj_danmu.groupby("ip_city").size().index
    for city in tqdm(list(citys)):
        contents = bjnvtj_danmu[bjnvtj_danmu["ip_city"] == city]["content"]
        texts = []
        for text in list(contents):
            texts.append(remove_words(text))
        save_wordcloud("".join(texts),path,city)
city_wordcloud("/Users/weduoo/Desktop/youku/")

## 评论数据分析

In [ ]:
comments = pd.read_csv("/Users/weduoo/Desktop/youku/bjnztj_comments.csv")
comments.head()

In [ ]:
def convert_user_city(userIds):
    user_city = []
    for uid in tqdm(userIds):
        city = bjnvtj_danmu[bjnvtj_danmu["uid"] == uid]
        if len(city) > 0:
            user_city.append(city["ip_city"].values[0])
        else:
            user_city.append("未知")
    return user_city
comments["city"] = convert_user_city(list(comments["userId"]))
comments.head()

In [ ]:
c_city = comments.groupby("city").size()
print(c_city.index)
print(c_city.values)

In [ ]:
upCount = comments[comments["downCount"] > 50].groupby("episode_cn")
def episode_wordcloud(path):
    episodes = upCount.size().index
    for episode in tqdm(list(episodes)):
        contents = comments[(comments["episode_cn"] == episode) & (comments["downCount"] > 50)]["content"]
        texts = []
        for text in list(contents):
            texts.append(remove_words(text))
        save_wordcloud("".join(texts),path,episode)
episode_wordcloud("/Users/weduoo/Desktop/youku/")

# 微博信息获取

## 信息获取说明
### 用户微博获取
微博 url：https://m.weibo.cn/api/container/getIndex?type=uid&value=1192329374&containerid=1076031192329374
微博 url(含页数)：https://m.weibo.cn/api/container/getIndex?type=uid&value=1192329374&containerid=1076031192329374&page=2

**参数：**
- type：类型， 用户id，[uid]
- value：类型的值，用户id， [1192329374]
- containerid：容器类型，107603 + 用户 id
- page: 页数，每页返回 10 条微博

**返回数据**

这里值获取微博信息不获取其他信息，因此在 `cards` 数组里只获取 `card_type` 类型为 `9` 的 cards。接着就是获取 mblog 里面的微博信息，需要后去如下的信息：
- id/idstr/mid: 当前微博的 id
- created_at: 在什么时间创建（不太有意义）
- text: 实际的微博，表情会与文本一起存在
- raw_text: 原微博，在是**转发**的微博时会有此属性
- source: 微博从什么平台发布
- reposts_count: 转发数量
- comments_count: 评论数量
- attitudes_count: 点赞数量

获取微博配的图片，图片在 `pics` 属性中，若是纯文字则没有图片。在最层的 `url` 中存储的是小图片，在 `large` 中存贮的是大图。

获取微博配的视频，视频在 `page_info` 中，之后在找到 `media_info` 里面的 `stream_url` 就是视频的地址。

若属性里还有 `retweeted_status` 就说明此条微博时评论引用微博。暂时不处理这种微博，只记录。
```json
{
    "ok": 1, 
    "data": {
        "cards": [
            {
                "card_type": 9, 
                "itemid": "1076031192329374_-_4237116843937317", 
                "scheme": "https://m.weibo.cn/status/GfzzSgwh7?mblogid=GfzzSgwh7&luicode=10000011&lfid=1076031192329374", 
                "mblog": {
                    "created_at": "2小时前", 
                    "id": "4237116843937317", 
                    "idstr": "4237116843937317", 
                    "mid": "4237116843937317", 
                    "can_edit": false, 
                    "text": "今天好孩子发布会，好像瘦点儿了<span class=\"url-icon\"><img src=\"//h5.sinaimg.cn/m/emoticon/icon/default/d_touxiao-3458a765a2.png\" style=\"width:1em;height:1em;\" alt=\"[偷笑]\"></span> 看着我拉着婴儿车旋转舞动会不会很嗨🤗 ​​​", 
                    "textLength": 75, 
                    "source": "", 
                    "favorited": false, 
                    "thumbnail_pic": "http://wx4.sinaimg.cn/thumbnail/4711809ely1fr2z4cfy47j21l6280e81.jpg", 
                    "bmiddle_pic": "http://wx4.sinaimg.cn/bmiddle/4711809ely1fr2z4cfy47j21l6280e81.jpg", 
                    "original_pic": "http://wx4.sinaimg.cn/large/4711809ely1fr2z4cfy47j21l6280e81.jpg", 
                    "is_paid": false, 
                    "mblog_vip_type": 0, 
                    "user": {}, 
                    "picStatus": "0:1,1:1,2:1,3:1,4:1,5:1", 
                    "reposts_count": 3343, 
                    "comments_count": 4917, 
                    "attitudes_count": 82175, 
                    "pending_approval_count": 0, 
                    "isLongText": false, 
                    "visible": {}, 
                    "more_info_type": 0, 
                    "content_auth": 0, 
                    "edit_config": {}, 
                    "mblogtype": 0, 
                    "weibo_position": 1, 
                    "bid": "GfzzSgwh7", 
                    "pics": [
                        {
                            "pid": "4711809ely1fr2z4cfy47j21l6280e81", 
                            "url": "https://wx4.sinaimg.cn/orj360/4711809ely1fr2z4cfy47j21l6280e81.jpg", 
                            "size": "orj360", 
                            "geo": {}, 
                            "large": {"url":""}
                        }
                    ]
                }, 
                "show_type": 0
            }
        ], 
        "showAppTips": 0, 
        "scheme": "sinaweibo://cardlist?containerid=1076031192329374&extparam=&luicode=10000011&lfid=1076031192329374&featurecode="
    }
}
```

### 获取微博的评论
获取评论方式 url：https://m.weibo.cn/api/comments/show?id=4236304998304332&page=1

参数：
- id: 当前微博 id，可以从微博请求连接中获取到
- page: 当前第几页

**返回数据**

这里要获取用户的评论和用户基本信息，所有的返回数据在 `data/data` 数组中需要获取如下信息：
- `source`: 发布的设备，
- `text`: 评论信息
- `like_counts`: 点赞数量
- `user` 字段中需要获取如下信息：
    - `id`: 用户的id，用于统计参与的用户
    - `profile_url`: 用户的首页信息地址
    - `screen_name`: 昵称

```json
{
    "ok": 1, 
    "msg": "数据获取成功", 
    "data": {
        "data": [
            {
                "id": 4237164545724845, 
                "created_at": "2分钟前", 
                "source": "前后2000万 OPPO R11s", 
                "user": {
                    "id": 6510497406, 
                    "screen_name": "风继续吹-201511", 
                    "profile_image_url": "https://tvax4.sinaimg.cn/crop.0.0.132.132.180/0076BnMGly8fpjoi3pd5bj303o03omx0.jpg", 
                    "verified": false, 
                    "verified_type": -1, 
                    "mbtype": 0, 
                    "profile_url": "https://m.weibo.cn/u/6510497406?uid=6510497406&luicode=10000011&lfid=1076031192329374", // 个人主页地址
                    "remark": "", 
                    "following": false, //我是否关注
                    "follow_me": false // 是否关注我
                }, 
                "text": "挺美的 辣妈 很美", // 评论
                "like_counts": 0,  // 点赞数量
                "liked": false // 我是否点赞
            }
        ], 
        "total_number": 5283, 
        "max": 529
    }
}
```

### 获取微博点赞数据 
点赞信息获取 url：https://m.weibo.cn/api/attitudes/show?id=4237116843937317&page=1

**参数：**
- `id`: 词条微博的 id
- `page`: 第几页，每页展示 50 条

**返回数据**

需要获取的数据在 `data/data` 字段中，需获取如下信息
- `source`: 操作设备名称
- `user`:用户信息
    - `id`: 用户 id
    - `profile_url`: 个人主页地址
    - `screen_name`: 昵称
```json
{
    "ok": 1, 
    "msg": "数据获取成功", 
    "data": {
        "total_number": 104951, 
        "max": 2100, 
        "data": [
            {
                "id": 4237168459296182, 
                "created_at": "刚刚", 
                "source": "柔光自拍vivo X7", 
                "user": {
                    "id": 5803297786, 
                    "screen_name": "Shirley-的主场", 
                    "profile_image_url": "https://tva1.sinaimg.cn/crop.0.0.438.438.180/006kK2Rcjw8ezfaiyje3qj30c60c6mxn.jpg", 
                    "verified": false, 
                    "verified_type": -1, 
                    "mbtype": 0, 
                    "profile_url": "https://m.weibo.cn/u/5803297786?uid=5803297786&luicode=10000011&lfid=1076031192329374", 
                    "remark": "", 
                    "following": false, 
                    "follow_me": false
                }
            }
        ]
    }
}
```

### 获取微博转发数据
微博转发信息获取 url：https://m.weibo.cn/api/statuses/repostTimeline?id=4237116843937317&page=1

**参数：**
- `id`: 此条微博的id
- `page`: 第几页，每页显示 10 条

**返回数据：**

返回的数据均在 `data/data` 字段下，此处要获取如下信息：
- `text`: 转发时的评论，如果内容为 "转发微博" 则表示值转发不评论
- `user`:用户信息
    - `id`: 用户 id
    - `profile_url`: 个人主页地址
    - `screen_name`: 昵称

```json
{
    "ok": 1, 
    "msg": "数据获取成功", 
    "data": {
        "data": [
            {
                "id": 4237170380036347, 
                "raw_text": "转发微博", 
                "text": "转发微博", 
                "created_at": "刚刚", 
                "user": {
                    "id": 3675989462, 
                    "screen_name": "海斗su", 
                    "profile_image_url": "https://tvax4.sinaimg.cn/crop.0.0.664.664.180/db1b25d6ly8fisne8026pj20ig0iggmq.jpg", 
                    "verified": false, 
                    "verified_type": -1, 
                    "mbtype": 2, 
                    "profile_url": "https://m.weibo.cn/u/3675989462?uid=3675989462&luicode=10000011&lfid=1076031192329374", 
                    "remark": "", 
                    "following": false, 
                    "follow_me": false
                }, 
                "like_counts": 0, 
                "liked": false
            }
        ], 
        "total_number": 3565, 
        "hot_total_number": 5, 
        "max": 357
    }
}

```

### 获取微博中评论、转发、点赞的用户的信息
获取用户信息：（super-帅比）https://m.weibo.cn/api/container/getIndex?uid=6013003248&luicode=10000011&lfid=1076036013003248&featurecode=20000320&containerid=1005056013003248

containerid 为：100505 + 用户 id

用户基础信息获取：[https://m.weibo.cn/api/container/getIndex?containerid=2302836013003248_-
_INFO&title=%25E5%259F%25BA%25E6%259C%25AC%25E8%25B5%2584%25E6%2596%2599&
luicode=10000011&lfid=2302836013003248&featurecode=20000320]

** 参数: ** 
- containerid 为：230283 + 用户 id
- lfid 为：230283 + 用户 id

**返回数据：**

所有的数据均在 `data/cards` 中，需要从每个 card 中获取到 `card_group`, 从中在获取到 `昵称``性别``所在地``简介``注册时间` 信息。
```json
{
    "ok": 1, 
    "data": {
        "cardlistInfo": {}, 
        "cards": [
            {
                "card_type": 11, 
                "show_type": 2, 
                "card_group": [
                    {
                        "card_type": 41, 
                        "item_name": "昵称", 
                        "item_content": "Dejlige_"
                    }, 
                    {
                        "card_type": 41, 
                        "item_name": "性别", 
                        "item_content": "女"
                    }, 
                    {
                        "card_type": 41, 
                        "item_name": "所在地", 
                        "item_content": "山西 运城"
                    }, 
                    {
                        "card_type": 41, 
                        "item_name": "简介", 
                        "item_content": "底线 蔡徐坤 ❤️"
                    }
                ]
            }, 
            {
                "card_type": 11, 
                "show_type": 2, 
                "is_asyn": 0, 
                "card_group": [ 
                    {
                        "card_type": 41, 
                        "item_name": "注册时间", 
                        "item_content": "2016-01-28"
                    }
                ]
            }
        ], 
        "showAppTips": 0, 
        "scheme": "sinaweibo://cardlist?containerid=2302835841808579_-
        _INFO&extparam=&luicode=10000011&lfid=2302835841808579&featurecode="
    }
}
```

In [ ]:
# 导入相关包
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from pyecharts import Bar
from pyecharts import online
import urllib
from urllib import request
import json
from retrying import retry
import os
import csv
from tqdm import tqdm
import time
import ssl
import requests

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# 让 pyecharts 在 jupyter 下载时有图片
online()

# ssl 设置
requests.packages.urllib3.disable_warnings()

## 信息获取代码

### 获取用户所发微博
将微博保存在 csv 文件中

In [ ]:
"""
下载图片
mid: 微博 id
pics: 待下载的图片
imagepath: 图片保存地址
""" 
def download_images(mid,pics,imagepath):
    for index,image_data in enumerate(pics):
        # 大图
        url = image_data["large"]["url"]
        #获得图片后缀
        file_suffix = os.path.splitext(url)[1]
        filename = imagepath  + str(mid) + "_" + str(index) + file_suffix
        if not os.path.exists(filename):
            print("下载..",url,filename)
            time.sleep(1)
            image = requests.get(url,verify=False)
            with open(filename, 'wb') as file:
                file.write(image.content)

In [ ]:
"""
获取用户微博，并保存到指定路径的指定文件
"""
def get_mblog(uid,filepath,imagepath,weibo_log):
    if not os.path.exists(imagepath):
        print(imagepath, "文件路径不存在")
        return
    url = "https://m.weibo.cn/api/container/getIndex?type=uid&value="+uid+"&containerid=107603"+uid+"&page="
    mblog_io = open(filepath,"a")
    mblog_csv = csv.writer(mblog_io)
    mblog_csv.writerow(["mid","created_at","text","source","reposts_count",\
                        "comments_count","attitudes_count","is_reposts"])
    
    with open(weibo_log) as f:
        mids = f.read().splitlines()
    logs = open(weibo_log,"a")
    CARRY_ON = True
    PAGE = 1
    while CARRY_ON:
        print(PAGE)
        try:
            rep = requests.get(url + str(PAGE),verify=False)
            data = rep.json()
            cards = data["data"]["cards"]
            if len(cards) > 0:
                for card in cards:
                    if card["card_type"] == 9 and card["mblog"]["mid"] not in mids:
                        mblog = card["mblog"]
                        mid = mblog["mid"]
                        
                        created_at = mblog["created_at"]
                        text = mblog["text"]
                        source = mblog["source"]
                        reposts_count = mblog["reposts_count"]
                        comments_count = mblog["comments_count"]
                        attitudes_count = mblog["attitudes_count"]
                        if "retweeted_status" in mblog.keys():
                            is_reposts = 1
                        else:
                            is_reposts = 0

                        mblog_csv.writerow((mid,created_at,text,source,reposts_count,\
                                            comments_count,attitudes_count,is_reposts))
                        # 获取图片
                        if "pics" in mblog.keys():
                            download_images(mid,mblog["pics"],imagepath)
                        logs.writelines(mid+"\n")
                time.sleep(3)                    
                PAGE += 1
            else:
                # 已无返回数据
                CARRY_ON = False
        except Exception:
            print("产生错误")
            mblog_io.close()
            logs.close()
            print("重新开始")
            get_mblog(uid,filepath,imagepath,weibo_log)
            raise
    mblog_io.close()
    logs.close()

## 获取谢娜的微博

In [ ]:
weibo_data = "/Users/weduoo/Desktop/weibo/xiena.csv"
image_path =  "/Users/weduoo/Desktop/weibo/xiena/"
weibo_logs = "/Users/weduoo/Desktop/weibo/xiena_complete.txt"
get_mblog("1192329374",weibo_data,image_path,weibo_logs)

## 获取走饭的微博

In [ ]:
weibo_data_zoufan = "/Users/weduoo/Desktop/weibo/zoufan.csv"
image_path_zoufan =  "/Users/weduoo/Desktop/weibo/zoufan/"
weibo_log_zoufan = "/Users/weduoo/Desktop/weibo/zoufan_complete.txt"
get_mblog("1648007681",weibo_data_zoufan,image_path_zoufan,weibo_log_zoufan)

## 下载图片

In [ ]:
weibo_data_pics = "/Users/weduoo/Desktop/weibo/pics.csv"
image_path_pics =  "/Users/weduoo/Desktop/weibo/pics/"
weibo_log_pics = "/Users/weduoo/Desktop/weibo/pics_complete.txt"
get_mblog("1304494805",weibo_data_pics,image_path_pics,weibo_log_pics)

## 获取最美中国风图片

In [ ]:
weibo_data_pics = "/Users/weduoo/Desktop/weibo/chinabeau.csv"
image_path_pics =  "/Users/weduoo/Desktop/weibo/chinabeau/"
weibo_log_pics = "/Users/weduoo/Desktop/weibo/china_complete.txt"
get_mblog("2142452271",weibo_data_pics,image_path_pics,weibo_log_pics)

## 获取老树画画的微博与图片

In [ ]:
weibo_data_pics = "/Users/weduoo/Desktop/weibo/laoshu.csv"
image_path_pics =  "/Users/weduoo/Desktop/weibo/laoshu/"
weibo_log_pics = "/Users/weduoo/Desktop/weibo/laoshu_complete.txt"
get_mblog("1454064140",weibo_data_pics,image_path_pics,weibo_log_pics)

## 获取 nico 的微博

In [ ]:
weibo_data_pics = "/Users/weduoo/Desktop/weibo/nico/nico.csv"
image_path_pics =  "/Users/weduoo/Desktop/weibo/nico/nico/"
weibo_log_pics = "/Users/weduoo/Desktop/weibo/nico/nico_complete.txt"
get_mblog("2109243041",weibo_data_pics,image_path_pics,weibo_log_pics)

In [ ]:
ssl._create_default_https_context = ssl._create_unverified_context
req = 'https://m.weibo.cn/api/container/getIndex?type=uid&value=1192329374&containerid=1076031192329374&page=2'
data = urllib.request.urlopen(req).read()
print(data)

In [ ]:
from requests.packages import urllib3
 
urllib3.disable_warnings()
response = requests.get(req, verify=False)
print(response.status_code)

# 词语图分析

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re
def create_wordcloud(text):
    # 用于支持中文的设置
    font = '/Library/Fonts/Songti.ttc'
    wordcloud = WordCloud(background_color="white",font_path=font,\
                          width=1000, height=1000, margin=2).generate(text)
    # 生成图片
    plt.figure(figsize=(14,14))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()

## 走饭微博词云图

In [ ]:
def zoufan_wordcloud():
    data = pd.read_csv("/Users/weduoo/Desktop/weibo/zoufanV1.csv")
    weibo = []
    for text in tqdm(list(data["text"])):
#         t = str(text).strip().replace("哈","").replace("呵","").replace("啦","").replace("啊","")
#         t = remove_words(str(text))
        text = re.sub('[a-zA-Z]','', str(text))
        weibo.append(str(text).strip())
    create_wordcloud("\n".join(weibo))
zoufan_wordcloud()

In [ ]:
a = "weduoo.com"
id(a)

In [ ]:
x=y=z=1

In [ ]:
print(x,y,z)

In [ ]:
a=b=["weduoo.com","hello"]
a.append("weduoo.cn")
print(a,b)

In [ ]:
import os
rPath = "/Users/weduoo/Desktop"
for path in os.listdir(rPath):
    sPath = os.path.join(rPath, path)
    if os.path.isdir(sPath):
        print(sPath)   

In [ ]:
b = dict(zip(('a','b','c','d','e'),(1,2,3,4,5)))
print(b)

In [ ]:
class A(object):
    def go(self):
        print("go A go!")
    def stop(self):
        print("stop A stop!")
    def pause(self):
        raise Exception("Not Implemented")

class B(A):
    def go(self):
        super(B, self).go()
        print("go B go!")

class C(A):
    def go(self):
        super(C, self).go()
        print("go C go!")
    def stop(self):
        super(C, self).stop()
        print("stop C stop!")

class D(B,C):
    def go(self):
        super(D, self).go()
        print("go D go!")
    def stop(self):
        super(D, self).stop()
        print("stop D stop!")
    def pause(self):
        print("wait D wait!")

class E(B,C): pass
a = A()
b = B()
c = C()
d = D()
e = E()
d.go()
D.mro()

In [ ]:
h=11122345
print(sys.getrefcount(11122345))

In [ ]:
import sys
a = 10103
init_cnt = sys.getrefcount(a) - 1

print ("init", init_cnt)
def func(c):
    print ('in function, count: ', sys.getrefcount(c) - 1)
    print ('函数调用，计数器增加', sys.getrefcount(c) - 1 - init_cnt)
func(a)

In [ ]:
id(a)

In [ ]:
hex(id(a))

# 清明上河图

In [3]:
import requests
import os
import sys
import time

In [ ]:
def download_qmsht_patches(dirs):
    pre = 115
    url = 'http://en.dpm.org.cn/tilegenerator/dest/files/image/8831/2007/1900/img0009_files/15/'
    for i in range(116):
        for j in range(6):
            name = str(i) + '_' + str(j) + '.jpg'
            filename = os.path.join(dirs,name)
            if not os.path.exists(filename):
                print("下载..",filename)
                time.sleep(0.1)
                image = requests.get(url+name,verify=False)
                with open(filename, 'wb') as file:
                    file.write(image.content)

In [ ]:
download_qmsht_patches('/Users/weduoo/Desktop/qmsht/')

## 图片拼接
### 纵向拼接
每一列有六个图片，第一张图片高 257 像素，中间四章高 258 像素，最后一张高 121 像素，总高 1410 像素。宽度每张图片均不一样。

In [19]:
import os
from PIL import Image

In [25]:
def lengthways(origin_dir,dest_dir):
    count = [0, 1, 2, 3, 4, 5]
    # 六张图的高度
    hs = [257, 258, 258, 258, 258, 121]
    # 总高
    HEIGHT = 1410
    for i in range(116):
        # 获取第一张图的宽度
        w = Image.open(os.path.join(origin_dir, str(i) + '_' + str(0) + '.jpg')).size[0]
        imagefile = []
        for j in count:
            name = str(i) + '_' + str(j) + '.jpg'
            imagefile.append(Image.open(os.path.join(orgin_dir, name))) 
        target = Image.new('RGB', (w, HEIGHT))
        x = 0
        y = 0
        for k,image in enumerate(imagefile):     
            target.paste(image, (0, y))
            y += hs[k] # 从上往下拼接，左上角的纵坐标递增
        quality_value = 100
        target.save(os.path.join(dest_dir,str(i)+'.jpg'), quality = quality_value) 

In [26]:
orgin = '/Users/weduoo/Desktop/qmsht/origin/'
dest = '/Users/weduoo/Desktop/qmsht/second/'
lengthways(orgin, dest)

### 横向拼接
高度固定为 1410 ，宽度需要根据实际计算出来

In [30]:
def crosswise(origin_dir, dest_dir):
    HEIGHT = 1410
    # 存储 116 张图的宽度
    ws = []
    # 存储 116 张图片，用于拼接
    images = []
    for i in range(116):
        img = Image.open(os.path.join(origin_dir, str(i)+'.jpg'))
        images.append(img)
        ws.append(img.size[0])
    # 计算总的宽度
    WIDTH = sum(ws)
    print(WIDTH)
    target = Image.new('RGB', (WIDTH, HEIGHT))
    x = 0
    y = 0
    for k,image in enumerate(images):     
        target.paste(image, (x, y))
        x += ws[k] # 从左往右拼接，y 不变，x 值增加
    quality_value = 100
    target.save(os.path.join(dest_dir,'qmsht.jpg'), quality = quality_value) 

In [31]:
orgin2 = '/Users/weduoo/Desktop/qmsht/second/'
dest2 = '/Users/weduoo/Desktop/qmsht/last/'
crosswise(orgin2, dest2)

29817


In [11]:
url = 'https://image.slidesharecdn.com/linearalgebrawithaplications-131026133441-phpapp02/95/'
       
for i in range(171,181):
    name = 'linear-algebra-with-aplications-steven-j-leon-'+str(i)+'-638.jpg?cb=1382794707'
    filename = os.path.join('/Users/weduoo/Desktop/download/', str(i) + '.jpg')
    if not os.path.exists(filename):
        print("下载..",i)
        time.sleep(0.1)
        image = requests.get(url+name,verify=True)
        with open(filename, 'wb') as file:
            file.write(image.content)

下载.. 171
下载.. 172
下载.. 173
下载.. 174
下载.. 175
下载.. 176
下载.. 177
下载.. 178
下载.. 179
下载.. 180
